<a href="https://colab.research.google.com/github/sarcasticvibes/Google-Quest-QnA-Labeling-via-BERT/blob/master/Google_Quest_Q%26A_labeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sarcasticvibes","key":"41883d4fbccff1f3662a136f9c271ee4"}'}

In [0]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [0]:
! kaggle competitions download -c 'google-quest-challenge'
! mkdir training_set

In [0]:
! unzip train.csv.zip -d training_set

In [0]:
!pip install transformers

In [0]:
!wget https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt

In [0]:
import os
import torch
import pandas as pd
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from torch.optim import lr_scheduler

from sklearn import model_selection
from sklearn import metrics
import transformers
import tokenizers
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
from tqdm.autonotebook import tqdm
from sklearn.model_selection import train_test_split
import utils
from scipy import stats

In [2]:
torch.cuda.get_device_name()

'Tesla P100-PCIE-16GB'

In [0]:
df = pd.read_csv('/content/training_set/train.csv')

In [16]:
df

,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,question_not_really_a_question,question_opinion_seeking,question_type_choice,question_type_compare,question_type_consequence,question_type_definition,question_type_entity,question_type_instructions,question_type_procedure,question_type_reason_explanation,question_type_spelling,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,0,What am I losing when using extension tubes in...,After playing around with macro photography on...,ysap,https://photo.stackexchange.com/users/1024,"I just got extension tubes, so here's the skin...",rfusca,https://photo.stackexchange.com/users/1917,http://photo.stackexchange.com/questions/9169/...,LIFE_ARTS,photo.stackexchange.com,1.000000,0.333333,0.000000,0.000000,0.000000,0.0,1.000000,1.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.0,0.000000,0.0,1.000000,0.000000,0.000000,0.0,1.000000,1.000000,0.666667,1.000000,1.000000,0.800000,1.000000,0.000000,0.000000,1.000000
1,1,What is the distinction between a city and a s...,I am trying to understand what kinds of places...,russellpierce,https://rpg.stackexchange.com/users/8774,It might be helpful to look into the definitio...,Erik Schmidt,https://rpg.stackexchange.com/users/1871,http://rpg.stackexchange.com/questions/47820/w...,CULTURE,rpg.stackexchange.com,1.000000,1.000000,0.000000,0.500000,1.000000,1.0,0.444444,0.444444,0.666667,0.0,0.000000,0.666667,0.666667,0.0,0.333333,0.0,0.000000,0.000000,0.333333,0.0,0.888889,0.888889,0.555556,0.888889,0.888889,0.666667,0.000000,0.000000,0.666667,0.888889
2,2,Maximum protusion length for through-hole comp...,I'm working on a PCB that has through-hole com...,Joe Baker,https://electronics.stackexchange.com/users/10157,Do you even need grooves? We make several pro...,Dwayne Reid,https://electronics.stackexchange.com/users/64754,http://electronics.stackexchange.com/questions...,SCIENCE,electronics.stackexchange.com,0.888889,0.666667,0.000000,1.000000,1.000000,1.0,0.666667,0.444444,0.333333,0.0,0.333333,0.000000,0.000000,0.0,0.000000,0.0,1.000000,0.333333,0.333333,0.0,0.777778,0.777778,0.555556,1.000000,1.000000,0.666667,0.000000,0.333333,1.000000,0.888889
3,3,Can an affidavit be used in Beit Din?,"An affidavit, from what i understand, is basic...",Scimonster,https://judaism.stackexchange.com/users/5151,"Sending an ""affidavit"" it is a dispute between...",Y e z,https://judaism.stackexchange.com/users/4794,http://judaism.stackexchange.com/questions/551...,CULTURE,judaism.stackexchange.com,0.888889,0.666667,0.666667,1.000000,1.000000,1.0,0.444444,0.444444,0.000000,0.0,0.000000,1.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.888889,0.833333,0.333333,0.833333,1.000000,0.800000,0.000000,0.000000,1.000000,1.000000
4,5,How do you make a binary image in Photoshop?,I am trying to make a binary image. I want mor...,leigero,https://graphicdesign.stackexchange.com/users/...,Check out Image Trace in Adobe Illustrator. \n...,q2ra,https://graphicdesign.stackexchange.com/users/...,http://graphicdesign.stackexchange.com/questio...,LIFE_ARTS,graphicdesign.stackexchange.com,1.000000,0.666667,0.000000,1.000000,1.000000,1.0,0.666667,0.666667,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,1.000000,0.000000,1.000000,0.0,1.000000,1.000000,0.666667,1.000000,1.000000,0.800000,1.000000,0.000000,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [0]:
class QnA_labelingDataset:
    def __init__(self, question_title, question_body, answer, labels):
        self.question_title = question_title
        self.question_body = question_body
        self.answer = answer
        self.labels = labels
        self.tokenizer = TOKENIZER
        self.max_len = MAX_LEN
    
    def __len__(self):
        return len(self.question_title)
    
    def __getitem__(self, item):
        question_title = str(self.question_title[item])
        question_body = str(self.question_title[item])
        answer = str(self.answer[item])
        labels = self.labels[item,:]

        inputs = self.tokenizer.encode_plus(question_title+' '+question_body,
                                           answer,
                                           add_special_tokens=True,
                                           max_length=self.max_len)
        input_ids = inputs['input_ids']
        token_type_ids = inputs['token_type_ids']
        mask = inputs['attention_mask']

        #print(len(token_type_ids))
        padding_length = self.max_len - len(token_type_ids)
        
        if padding_length > 0:
          input_ids = input_ids + ([0] * padding_length)
          mask = mask + ([0] * padding_length)
          token_type_ids = token_type_ids + ([0] * padding_length)

        return {'ids': torch.tensor(input_ids, dtype=torch.long),
                'mask': torch.tensor(mask, dtype=torch.long),
                'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
                'targets': torch.tensor(labels, dtype=torch.float)}

In [0]:
class QnA_labelingModel(transformers.BertPreTrainedModel):
    def __init__(self, conf):
        super(QnA_labelingModel, self).__init__(conf)
        self.bert = transformers.BertModel.from_pretrained('bert-base-uncased', config=conf)
        self.drop_out = nn.Dropout(0.2)
        self.output_layer = nn.Linear(768, 30)
        torch.nn.init.normal_(self.output_layer.weight, std=0.02)
    
    def forward(self, ids, mask, token_type_ids):
        _, out = self.bert(ids, attention_mask=mask, token_type_ids=token_type_ids)

        out = self.drop_out(out)
        logits = self.output_layer(out)
        
        return logits

In [0]:
def loss_fn(outputs, targets):
  return nn.BCEWithLogitsLoss()(outputs, targets)

In [0]:
def train_fn(data_loader, model, optimizer, device, scheduler=None):
    model.train()
    losses = utils.AverageMeter()
    tk0 = tqdm(data_loader, total=len(data_loader))
    
    for _, d in enumerate(tk0):

        ids = d["ids"]
        token_type_ids = d["token_type_ids"]
        mask = d["mask"]
        targets = d["targets"]

        ids = ids.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        targets = targets.to(device, dtype=torch.float)

        model.zero_grad()
        outputs = model(ids=ids, mask=mask, token_type_ids=token_type_ids,)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        scheduler.step()

        losses.update(loss.item(), ids.size(0))
        tk0.set_postfix(loss=losses.avg)

In [0]:
def train():
    dfx = pd.read_csv('/content/training_set/train.csv').fillna('none')

    df_train, df_valid = train_test_split(dfx, test_size=.1, random_state=42)

    df_train = df_train.reset_index(drop=True)
    df_valid = df_valid.reset_index(drop=True)

    sample = pd.read_csv('sample_submission.csv')
    target_columns = list(sample.drop('qa_id', axis=1).columns)
    train_targets = df_train[target_columns].values
    valid_targets = df_valid[target_columns].values
    
    train_dataset = QnA_labelingDataset(question_title=df_train['question_title'],
                                        question_body=df_train['question_body'], 
                                        answer=df_train['answer'],
                                        labels=train_targets)

    train_data_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=TRAIN_BATCH_SIZE,
        num_workers=4
    )

    valid_dataset = QnA_labelingDataset(question_title=df_valid['question_title'],
                                        question_body=df_valid['question_body'], 
                                        answer=df_valid['answer'],
                                        labels=valid_targets)

    valid_data_loader = torch.utils.data.DataLoader(
        valid_dataset,
        batch_size=VALID_BATCH_SIZE,
        num_workers=2
    )

    device = torch.device("cuda")
    model_config = transformers.BertConfig.from_pretrained('bert-base-uncased')
    model = QnA_labelingModel(conf=model_config)
    model.to(device)

    num_train_steps = int(len(df_train) / TRAIN_BATCH_SIZE * EPOCHS)
    param_optimizer = list(model.named_parameters())
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    optimizer_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.001},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
    ]
    optimizer = AdamW(optimizer_parameters, lr=LEARNING_RATE)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, 
        num_warmup_steps=0, 
        num_training_steps=num_train_steps
    )

    es = utils.EarlyStopping(patience=2, mode="max")
    print(f"Training is Starting:")
    
    for epoch in range(EPOCHS):
        train_fn(train_data_loader, model, optimizer, device, scheduler=scheduler)
        outputs, targets = eval_fn(valid_data_loader, model, device)

        spearman_score = []
        for i in range(outputs.shape[1]):
          p1 = list(targets[:, i])
          p2 = list(outputs[:, i])
          coef, _ = np.nan_to_num(stats.spearmanr(p1, p2))
          spearman_score.append(coef)
        score = np.mean(spearman_score)

        print(f"Score = {score}")
        es(score, model, model_path=f"model.bin")
        if es.early_stop:
            print("Early stopping")
            break

In [0]:
def eval_fn(data_loader, model, device):
    model.eval()
    losses = utils.AverageMeter()

    output_list = []
    target_list = []

    with torch.no_grad():
        tk0 = tqdm(data_loader, total=len(data_loader))
        for _, d in enumerate(tk0):
            ids = d["ids"]
            token_type_ids = d["token_type_ids"]
            mask = d["mask"]
            targets = d["targets"]

            ids = ids.to(device, dtype=torch.long)
            token_type_ids = token_type_ids.to(device, dtype=torch.long)
            mask = mask.to(device, dtype=torch.long)
            targets = targets.to(device, dtype=torch.float)

            model.zero_grad()
            outputs = model(ids=ids, mask=mask, token_type_ids=token_type_ids,)
            loss = loss_fn(outputs, targets)
            losses.update(loss.item(), ids.size(0))

            output_list.append(outputs.cpu().detach().numpy())
            target_list.append(targets.cpu().detach().numpy())

            tk0.set_postfix(loss=losses.avg)

    return np.vstack(output_list), np.vstack(target_list)

In [0]:
MAX_LEN = 512
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 8
EPOCHS = 5
TOKENIZER = transformers.BertTokenizer.from_pretrained('bert-base-uncased')
LEARNING_RATE = 3e-5

In [11]:
train()

Training is Starting:



Score = 0.2550456013972514
Validation score improved (-inf --> 0.2550456013972514). Saving model!



Score = 0.29343999624796985
Validation score improved (0.2550456013972514 --> 0.29343999624796985). Saving model!



Score = 0.31347687537743824
Validation score improved (0.29343999624796985 --> 0.31347687537743824). Saving model!



Score = 0.3247118057790586
Validation score improved (0.31347687537743824 --> 0.3247118057790586). Saving model!



Score = 0.32741386738528966
Validation score improved (0.3247118057790586 --> 0.32741386738528966). Saving model!


In [0]:
df = pd.read_csv('test.csv')

In [13]:
df

,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com
...,...,...,...,...,...,...,...,...,...,...,...
471,9569,change gb4e enumeration in only one chapter,I am using gb4e for glossing examples in my th...,ToddAO,https://tex.stackexchange.com/users/28332,You can simply reset the exx counter just afte...,karlkoeller,https://tex.stackexchange.com/users/27635,http://tex.stackexchange.com/questions/165120/...,TECHNOLOGY,tex.stackexchange.com
472,9590,All anagrams in a File,Source : Microsoft Interview Question\n\nWe ar...,Spandan,https://stackoverflow.com/users/2426101,Slightly different approach from the one above...,Algorithmatic,https://stackoverflow.com/users/1122229,http://stackoverflow.com/questions/16872513/al...,STACKOVERFLOW,stackoverflow.com
473,9597,SQL Server: Time Series Table Design,I am creating a time series database that foll...,Jay Michael,https://dba.stackexchange.com/users/47001,A foreign key can reference a unique constrain...,mustaccio,https://dba.stackexchange.com/users/23721,http://dba.stackexchange.com/questions/105287/...,TECHNOLOGY,dba.stackexchange.com
474,9623,Is a countered flashback spell exiled?,If I flashback Past in Flames and it is counte...,BolasStone,https://boardgames.stackexchange.com/users/9981,If you counter a spell cast for its Flashback ...,murgatroid99,https://boardgames.stackexchange.com/users/2537,http://boardgames.stackexchange.com/questions/...,CULTURE,boardgames.stackexchange.com


In [0]:
sample = pd.read_csv('sample_submission.csv')
df = df.merge(sample)

In [15]:
device = torch.device("cuda")
model_config = transformers.BertConfig.from_pretrained('bert-base-uncased')

model = QnA_labelingModel(conf=model_config)
model.to(device)
model.load_state_dict(torch.load("model.bin"))
model.eval()

QnA_labelingModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=

In [0]:
sample = pd.read_csv('sample_submission.csv')
target_columns = list(sample.drop('qa_id', axis=1).columns)

In [52]:
final_output = []
test_targets = df[target_columns].values
test_dataset = QnA_labelingDataset(question_title=df['question_title'],
                                        question_body=df['question_body'], 
                                        answer=df['answer'],
                                        labels=test_targets)

data_loader = torch.utils.data.DataLoader(
    test_dataset,
    shuffle=False,
    batch_size=VALID_BATCH_SIZE,
    num_workers=1
)

with torch.no_grad():
    tk0 = tqdm(data_loader, total=len(data_loader))
    for bi, d in enumerate(tk0):
        ids = d["ids"]
        token_type_ids = d["token_type_ids"]
        mask = d["mask"]
        targets = d['targets']
        ids = ids.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        targets = targets.to(device, dtype=torch.float)

        outputs = model(ids=ids, mask=mask, token_type_ids=token_type_ids)

        outputs = torch.sigmoid(outputs).cpu().detach().numpy()
        for output in outputs:
          final_output.append(output)

In [0]:
final_output = np.array(final_output)

In [54]:
final_output.shape

(476, 30)

In [0]:
df.loc[:,target_columns] = final_output

In [56]:
df

,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,question_not_really_a_question,question_opinion_seeking,question_type_choice,question_type_compare,question_type_consequence,question_type_definition,question_type_entity,question_type_instructions,question_type_procedure,question_type_reason_explanation,question_type_spelling,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com,0.942073,0.720195,0.242755,0.739503,0.536681,0.550612,0.676432,0.634370,0.294637,0.008935,0.723553,0.815623,0.062085,0.028060,0.031732,0.038634,0.051201,0.067011,0.556536,0.010618,0.923467,0.926887,0.679421,0.964932,0.970427,0.876645,0.063482,0.060116,0.780562,0.942422
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com,0.884983,0.553177,0.011430,0.741781,0.766352,0.886405,0.531727,0.428006,0.148026,0.009903,0.438015,0.356679,0.014852,0.010619,0.011287,0.045675,0.858969,0.227402,0.068310,0.007060,0.725597,0.930225,0.635935,0.956193,0.966800,0.858434,0.880177,0.172130,0.080223,0.904929
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com,0.925729,0.717177,0.025098,0.829847,0.909191,0.935188,0.609637,0.547324,0.338081,0.005586,0.200790,0.767041,0.026572,0.017366,0.021710,0.026930,0.119038,0.073459,0.589779,0.007800,0.877588,0.942458,0.646845,0.969658,0.976537,0.875619,0.101933,0.062367,0.818392,0.935371
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com,0.841247,0.491606,0.024489,0.657764,0.761973,0.880738,0.543132,0.457071,0.177030,0.009576,0.490900,0.110993,0.012829,0.014528,0.011655,0.012768,0.718978,0.197323,0.707146,0.006172,0.706733,0.945502,0.676875,0.964140,0.967207,0.863251,0.708088,0.188442,0.795931,0.892360
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com,0.925126,0.428445,0.025757,0.755030,0.743148,0.799100,0.655185,0.503179,0.337146,0.007192,0.658253,0.510981,0.033269,0.010959,0.007494,0.043141,0.345601,0.157502,0.396627,0.005829,0.784948,0.921690,0.666402,0.970130,0.974648,0.845619,0.262164,0.103327,0.633822,0.934834
...,...,...,.

In [0]:
df.to_csv('/content/drive/My Drive/Google Quest:QnA Labeling/BERT result/reult.csv')

In [0]:
torch.save(model.state_dict(), '/content/drive/My Drive/Google Quest:QnA Labeling/BERT result/model.bin')